In [ ]:
import mysql.connector
from mysql.connector import Error

mydb = mysql.connector.connect(
 host="localhost",
 user="root",
 password=""
)
print(mydb)
mycursor = mydb.cursor(buffered=True)

In [4]:
mycursor.execute("create database choco_crunch")

In [ ]:
import pandas as pd
#1.Load dataset

df = pd.read_excel("Cleaned_chococrunch_with_points.xlsx")


# Create tables

import mysql.connector
from mysql.connector import Error

mydb = mysql.connector.connect(
 host="localhost",
 user="root",
 password="",
 database = "choco_crunch"
)
print(mydb)
mycursor = mydb.cursor(buffered=True)


mycursor.execute('''CREATE TABLE IF NOT EXISTS product_info (product_code VARCHAR(100) PRIMARY KEY,product_name TEXT,brand TEXT);''')


mycursor.execute("""CREATE TABLE IF NOT EXISTS nutrient_info (
                    product_code VARCHAR(100) PRIMARY KEY,
                    energy_kcal_value REAL,
                    energy_kj_value REAL,
                    carbohydrates_value REAL,
                    sugars_value REAL,
                    fat_value REAL,
                    saturated_fat_value REAL,
                    proteins_value REAL,
                    fiber_value REAL,
                    salt_value REAL,
                    sodium_value REAL,
                    fruits_vegetables_nuts_estimate_from_ingredients_100g REAL,
                    nutrition_score_fr INTEGER,
                    nova_group INTEGER,
                    FOREIGN KEY (product_code) REFERENCES product_info(product_code));""")


mycursor.execute("""CREATE TABLE IF NOT EXISTS derived_metrics (
                    product_code VARCHAR(100) PRIMARY KEY,
                    sugar_to_carb_ratio REAL,
                    calorie_category TEXT,
                    sugar_category TEXT,
                    is_ultra_processed TEXT,
                    FOREIGN KEY (product_code) REFERENCES product_info(product_code));""")

# Commit changes and close connection
mydb.commit()
mydb.close()

In [ ]:
# Insert data into tables

import mysql.connector
from mysql.connector import Error

mydb = mysql.connector.connect(
 host="localhost",
 user="root",
 password="",
 database = "choco_crunch"
)
print(mydb)
mycursor = mydb.cursor(buffered=True)

for index, row in df.iterrows():
        mycursor.execute('''INSERT INTO product_info 
                         (product_code, product_name, brand)VALUES (%s, %s, %s)''',
                         (row['code'], row['product_name'], row['brands']))

        

        mycursor.execute('''INSERT INTO nutrient_info
                         (product_code, energy_kcal_value, energy_kj_value, carbohydrates_value,
                         sugars_value, fat_value, saturated_fat_value, proteins_value, fiber_value,
                         salt_value, sodium_value, fruits_vegetables_nuts_estimate_from_ingredients_100g,
                         nutrition_score_fr, nova_group)VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''',
                         (row['code'], row['energy-kcal_value'], row['energy-kcal_value_computed'], row['carbohydrates_value'], 
                          row['sugars_value'], row['fat_value'], row['saturated-fat_value'], row['proteins_value'],
                          row['fiber_value'], row['salt_value'], row['sodium_value'],
                          row['fruits-vegetables-nuts-estimate-from-ingredients_100g'],
                          row['nutrition-score-fr'], row['nova-group']))


        mycursor.execute('''INSERT INTO derived_metrics
                         (product_code, sugar_to_carb_ratio, calorie_category, sugar_category, is_ultra_processed)
                         VALUES (%s, %s, %s, %s, %s)''',
                         (row['code'], row['sugar_to_carb_ratio'], row['calorie_category'],
                          row['sugar_category'], row['is_ultra_processed']))


# Commit changes and close connection
mydb.commit()
mydb.close()